In [91]:
import numpy as np
import pandas as pd 
import re
import random
import scipy

def makedf ():
    global NUM_BLOCK_INPUT, NUM_BLOCK, RANDOM_PICK, AXIS_RATIO, PARENT_RATIO, FP_RATIO, INPUT_DIR, OUTPUT_DIR, mixture_answer, membership_answer, samplename_dict, DEPTH_CUTOFF
    global  df, inputdf, input_containpos, np_vaf, membership, mutation_id, depth_list, NUM_MUTATION, samplename_dict
    global membership_answer, mutation_id, mixture_answer

    input_containpos = pd.read_csv(INPUT_DIR,  header = None, names =["pos", "sample", "info"], sep = "\t") 
    input_containpos ["cha1"] = "exclusive"  # monoclone이면 exclusive, 둘 이상의 clone이 합쳐진거면 parent
    input_containpos ["cha2"] = "space"       # 축 상에 있으면 axis, 공간 상에 있으면 space
    samplename_dict = {}
    NUM_MUTATION = input_containpos.shape[0]

    np_vaf = np.zeros((NUM_MUTATION, NUM_BLOCK_INPUT), dtype = 'float')
    inputdf = pd.DataFrame (np.zeros((NUM_MUTATION, NUM_BLOCK_INPUT), dtype = 'object'), columns = ['block' + str(i + 1) for i in range(NUM_BLOCK_INPUT)])
    mutation_id = []
    membership = []
    depth_list = []
    
    # input 형식은 n * 3 으로 구성 :   ID (chr_pos), membmership(정답 set 일 경우),  NUM_BLOCK_INPUT(3)만큼의 depth, alt 정보

    for row in range(NUM_MUTATION):
        mutation_id.append( str(input_containpos.iloc[row][0]) )            # "pos"
        membership.append( str(input_containpos.iloc[row][1]) )           # "sample"
        if "," in str(input_containpos.iloc[row][1]) :
            input_containpos.loc[row,"cha1"] = "parent"

        if str(input_containpos.iloc[row][1]) not in samplename_dict.keys():
            samplename_dict[str(input_containpos.iloc[row][1])] = int (len(samplename_dict))      # {'other': 0, 'V5': 1, 'V3': 2, 'V1': 3}           # 각 sample name을 숫자화시킴

        #rmv_bracket = re.sub("[\[\] ]", '', str(input_containpos.iloc[row][2])).split(",")            # [194, 25, 193, 66, 0, 0] 라고 되어 있는데 bracket과 한 칸 공백을 지움
        rmv_bracket=input_containpos.iloc[row][2].split(",")
        depth_row=[]
        for i in range(0, len(rmv_bracket), 2 ):
            depth = int(rmv_bracket[i])
            alt = int(rmv_bracket[i+1])
            ref = depth - alt

            col = int(i / 2)

            if depth == 0:
                np_vaf[row][col] = 0
                inputdf.iloc[row][col] = "0:0:0"
            else:    
                np_vaf[row][col] = round (alt / depth , 2)
                inputdf.iloc[row][col] = str(depth) + ":" + str(ref) + ":" + str(alt)
                depth_row.append(depth)

        # "0.0.0"을 그대로 놔둘 수 없다.  평균 depth로 갈음해서 바꿔 넣는다  (alt는 0으로 유지)
        for  i in range(0, len(rmv_bracket), 2 ):
            col = int(i / 2)
            if inputdf.iloc[row][col] == "0:0:0":
                inputdf.iloc[row][col] = str(round(np.mean(depth_row))) + ":" + str(round(np.mean(depth_row))) + ":0"
                input_containpos.loc[row,"cha2"] = "axis"
        
        

        depth_list.append(np.mean(depth_row))

    df = [[None] * NUM_BLOCK for i in range(inputdf.shape[0])]
    for row in range (inputdf.shape[0]):
        for col in range (NUM_BLOCK):
            df[row][col] = {"depth":int(inputdf.iloc[row][col].split(":")[0]), "ref":int(inputdf.iloc[row][col].split(":")[1]), "alt":int(inputdf.iloc[row][col].split(":")[2])}
            if df[row][col]["depth"] == 0:
                print (df[row][col], row, col)

    if len ([i for i in range (0, NUM_MUTATION) if ( ( "parent" in input_containpos.loc[i,"cha1"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]) / RANDOM_PICK < PARENT_RATIO:               # input 받은 PARENT_RATIO보다 내 database에서 적게 보유하고 있으면
        PARENT_RATIO = round( len ([i for i in range (0, NUM_MUTATION) if ( ( "parent" in input_containpos.loc[i,"cha1"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ])  / RANDOM_PICK, 3)
    if len ([i for i in range (0, NUM_MUTATION) if ( ( "axis" in input_containpos.loc[i,"cha2"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]) / RANDOM_PICK < AXIS_RATIO:               # input 받은 AXIS_RATIO보다 내 database에서 적게 보유하고 있으면
        AXIS_RATIO = round( len ([i for i in range (0, NUM_MUTATION) if ( ( "axis" in input_containpos.loc[i,"cha2"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]) / RANDOM_PICK, 3)
    if len ([i for i in range (0, NUM_MUTATION) if ( ( "FP" in input_containpos.loc[i,"sample"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]) / RANDOM_PICK < FP_RATIO:
        FP_RATIO = round( len ([i for i in range (0, NUM_MUTATION) if ( ( "FP" in input_containpos.loc[i,"sample"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]) / RANDOM_PICK, 3)

In [92]:
def random_pick_fun():
    global NUM_BLOCK_INPUT, NUM_BLOCK, RANDOM_PICK, AXIS_RATIO, PARENT_RATIO, FP_RATIO, INPUT_DIR, OUTPUT_DIR, mixture_answer, membership_answer, DEPTH_CUTOFF
    global  df, inputdf, input_containpos, np_vaf, membership, mutation_id, depth_list, NUM_MUTATION, NUM_CLONE, NUM_BLOCK, samplename_dict
    global membership_answer, mutation_id, mixture_answer

    # RANDOM하게 1000개만 뽑기. FP 의 비율도 조절해서...

    FP_index = [i for i in range (0, NUM_MUTATION) if ( (membership[i] == 'FP') & (depth_list[i] > DEPTH_CUTOFF) ) ]        # Depth 너무 낮으면 곤란하니 input을 받자
    PARENT_index = [i for i in range (0, NUM_MUTATION) if ( ( "parent" in input_containpos.loc[i,"cha1"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]        #

    CHILD_index = [i for i in range (0, NUM_MUTATION) if ( (membership[i] != 'FP')  &  ( "exclusive" in input_containpos.loc[i,"cha1"] )  & (depth_list[i] > DEPTH_CUTOFF) ) ]        # depth_list : 해당 mutation의 평균 depth        #
    AXIS_index = [i for i in range (0, NUM_MUTATION) if ( ( "axis" in input_containpos.loc[i,"cha2"] ) & (depth_list[i] > DEPTH_CUTOFF) ) ]        #

    CHILD_SPACE_index = list(set(CHILD_index) - set(AXIS_index))      # CHILD 이면서 SPACE에 위치해 있는 것들
    CHILD_AXIS_index = list(set(CHILD_index) & set(AXIS_index))       # CHILD 이면서  AXIS에 위치해 있는 것들

    FP_randomsample = random.sample(FP_index, int(RANDOM_PICK * (FP_RATIO)))
    PARENT_randomsample = random.sample(PARENT_index, int(RANDOM_PICK * (PARENT_RATIO)))
    CHILD_SPACE_randomsample = random.sample(CHILD_SPACE_index, int((1 - (FP_RATIO + PARENT_RATIO)) * (1 - AXIS_RATIO) * RANDOM_PICK ))
    CHILD_AXIS_randomsample = random.sample(CHILD_SPACE_index, int((1 - (FP_RATIO + PARENT_RATIO)) * (AXIS_RATIO) * RANDOM_PICK ))

    print ("FP 개수 : {}\tPARENT 개수 : {}\tCHILD_SPACE 개수 : {}\tCHILD_AXIS 개수 : {}".format(len(FP_randomsample), len (PARENT_randomsample), len (CHILD_SPACE_randomsample), len (CHILD_AXIS_randomsample)))

    random_index = sorted( FP_randomsample + PARENT_randomsample + CHILD_SPACE_randomsample + CHILD_AXIS_randomsample )             # 다 합치면 RADOM_PICK 개수가 되겠지

    # RANDOM_PICK 개만으로 재정비
    input_containpos =  input_containpos.iloc[random_index]
    inputdf  = inputdf.iloc[random_index]
    df = [df[i] for i in random_index]
    np_vaf = np_vaf[random_index]
    membership_answer = [membership[i] for i in random_index]
    mutation_id = [mutation_id[i] for i in random_index]

    #np_vaf + membership를 df 형식으로 하고 RANDOM_PICK개만 출력 
    t = pd.DataFrame(np_vaf, columns = ["block{0}".format(i) for i in range(NUM_BLOCK_INPUT)], index = mutation_id)
    t["membership_answer"] = pd.Series(membership_answer, index = mutation_id)
    t.to_csv (INPUT_DIR + "sampling_{0}.txt".format(RANDOM_PICK), index = True, header=True, sep = "\t")


    NUM_CLONE = len(set(membership_answer))
    mixture_answer = np.zeros ((NUM_BLOCK, NUM_CLONE), dtype = 'float')     #mixture 값을 일단 초기화
    for i in range(NUM_BLOCK):
        for j in range(NUM_CLONE):
            #print (i, j, list(set(membership_answer))[j])
            mixture_answer[i][j] = round(np.mean(np_vaf[[x  for x in range(len(membership_answer)) if membership_answer[x] == list(set(membership_answer))[j]]][:,i] * 2), 2)

In [89]:
def pyclone_dataset():
    for col in range (NUM_BLOCK):
        PYCLONE_OUTPUT="/data/project/Alzheimer/EM_cluster/old/pilot/04.EM_input/pyclone/block" + str(col) + ".tsv"
        with open (PYCLONE_OUTPUT, "w", encoding = "utf8") as output_pyclone:
            print ("\t".join(["mutation_id", "ref_counts", "var_counts", "normal_cn", "minor_cn", "major_cn"]), file = output_pyclone)
            for row in range(inputdf.shape[0]):
                pyclone_row = [mutation_id[row] , str(df[row][col]["ref"]), str(df[row][col]["alt"]), str(2), str(1), str(1)]
                print("\t".join(pyclone_row), file = output_pyclone)

def pyclonevi_dataset():
    PYCLONE_VI_OUTPUT="/data/project/Alzheimer/EM_cluster/old/pilot/04.EM_input/pyclone_vi/pyclone_vi_220610.tsv"
    with open (PYCLONE_VI_OUTPUT, "w", encoding = "utf8") as output_pyclone_vi:
        print ("\t".join(["mutation_id", "sample_id", "ref_counts", "alt_counts", "normal_cn", "major_cn", "minor_cn", "tumour_content"]), file = output_pyclone_vi)
        for row in range(inputdf.shape[0]):
            for col in range(NUM_BLOCK):
                pyclone_vi_row = [mutation_id[row] , "block" + str(col), str(df[row][col]["ref"]), str(df[row][col]["alt"]), str(2), str(1), str(1), str(1.0)]
                print("\t".join(pyclone_vi_row), file = output_pyclone_vi)


In [93]:
def main (**kwargs):
    global NUM_BLOCK_INPUT, NUM_BLOCK, RANDOM_PICK, AXIS_RATIO, PARENT_RATIO, FP_RATIO, INPUT_DIR, OUTPUT_DIR, mixture_answer, membership_answer, DEPTH_CUTOFF
    global  df, inputdf, input_containpos, np_vaf, membership, mutation_id, depth_list, NUM_MUTATION, NUM_CLONE, NUM_BLOCK, samplename_dict
    global membership_answer, mutation_id, mixture_answer

    NUM_BLOCK_INPUT = kwargs["NUM_BLOCK_INPUT"] 
    NUM_BLOCK = kwargs["NUM_BLOCK"] 
    RANDOM_PICK = kwargs["RANDOM_PICK"]
    AXIS_RATIO = kwargs["AXIS_RATIO"]
    PARENT_RATIO = kwargs["PARENT_RATIO"]
    FP_RATIO = kwargs["FP_RATIO"]
    DEPTH_CUTOFF = kwargs["DEPTH_CUTOFF"]
    INPUT_TSV = kwargs["INPUT_TSV"]
    INPUT_DIR = "/data/project/Alzheimer/EM_cluster/EM_input/MRS_" + str(NUM_BLOCK_INPUT) + "_sample/" + str(INPUT_TSV)
    OUTPUT_DIR ="./output/"

    print ("PARENT_RATIO = {}, FP_RATIO = {}, AXIS_RATIO = {}".format( PARENT_RATIO, FP_RATIO, AXIS_RATIO) )
    makedf()
    print ("PARENT_RATIO = {}, FP_RATIO = {}, AXIS_RATIO = {}".format( PARENT_RATIO, FP_RATIO, AXIS_RATIO) )
    random_pick_fun()
    # pyclone_dataset()
    # pyclonevi_dataset()

    #return (inputdf, df, np_vaf, membership_answer, mixture_answer,  mutation_id, samplename_dict, len(FP_randomsample), len (PARENT_randomsample), len (CHILD_SPACE_randomsample), len (CHILD_AXIS_randomsample))

kwargs = {"INPUT_TSV":"M1-4_M1-7_input.txt", "NUM_BLOCK_INPUT": 2, "NUM_BLOCK": 2, "NUM_CLONE_TRIAL_START" : 3, "NUM_CLONE_TRIAL_END" : 5, 
"RANDOM_PICK":2000, "AXIS_RATIO":0.3, "PARENT_RATIO": 0.2, "FP_RATIO":0.2, "TRIAL_NO" : 5, "DEPTH_CUTOFF" : 50}
#inputdf, df, np_vaf, membership_answer, mixture_answer, mutation_id, samplename_dict = main(**kwargs)

main(**kwargs)

PARENT_RATIO = 0.2, FP_RATIO = 0.2, AXIS_RATIO = 0.3
PARENT_RATIO = 0.044, FP_RATIO = 0.092, AXIS_RATIO = 0.3
184 184
FP 개수 : 184	PARENT 개수 : 88	CHILD_SPACE 개수 : 1209	CHILD_AXIS 개수 : 518
